<a href="https://colab.research.google.com/github/jermwatt/morphi_lab/blob/3d_tests/shape_e_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/openai/shap-e.git /content/shap-e

Cloning into '/content/shap-e'...
remote: Enumerating objects: 325, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 325 (delta 30), reused 15 (delta 15), pack-reused 278
Receiving objects: 100% (325/325), 11.71 MiB | 6.71 MiB/s, done.
Resolving deltas: 100% (47/47), done.


In [20]:
# cannot seem to install in edit mode via : pip install . -e
!cd shap-e && pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/shap-e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-u2zt6ed7/clip_bb44550be26a4171ab6709c9f60e292d
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-u2zt6ed7/clip_bb44550be26a4171ab6709c9f60e292d
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py) ... done
  Created wheel for shap-e: filename=shap_e-0.0.0-py3-none-any.whl size=133185 sha256=e00ce8366240d7f3ee804c94950556bce12fb24af047bf4e67c3536130197b3a
  Stored in directory: /root/.cache/pip/wheels/b5/4e/6c/3404b8a878380ed3136fc20610b783bc6dbdb95ce9625cc62b
Successfully built shap-e
  Attempting uninstall: shap-e
    Found existing installation: shap-e 0.0.0
    Uninstalling shap-e-0.0.0:
      Successfully uninstalled shap-e

In [18]:
import shap_e

In [19]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget
from shap_e.util.image_util import load_image

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [22]:
xm = load_model('transmitter', device=device)
model = load_model('image300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

100%|████████████████████████████████████████| 890M/890M [00:07<00:00, 132MiB/s]


  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

In [23]:
# pull some images from the web
import urllib.request

def download_file(url, output_path):
    urllib.request.urlretrieve(url, output_path)

# pull segmentation module
url = 'https://github.com/openai/shap-e/blob/main/shap_e/examples/example_data/corgi.png?raw=true'
local_path = "/content/corgi.png"
download_file(url,local_path)

In [24]:
batch_size = 4
guidance_scale = 3.0

# To get the best result, you should remove the background and show only the object of interest to the model.
image = load_image("/content/corgi.png")

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(images=[image] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [ ]:
render_mode = 'nerf' # you can change this to 'stf' for mesh rendering
size = 512 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

HTML(value='<img src="data:image/gif;base64,R0lGODlhAAIAAocAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAADMAAG…